In [1]:
# K. Whiteaker 2024-05
# kwhit@mit.edu
# k8whiteaker@gmail.com

This code describes the flow of I-129 from production in spent fuel ("Source Term") to three possible back-end scenarios:

(1) Spent fuel is sent directly to HLW

(2) Spent fuel is reprocessed in a model based on La Hague where 96% of the I-129 is captured in liquid effluent and 4% is filtered

(3) Spent fuel is reprocessed in an idealized model where 100% of the I-129 is filtered

# I-129 Source Term

## Step 1: Calculate NPP fuel requirement

### VERSION A: Rough estimate (depreciated)

The cell below estimates the fuel requirement per GWe-yr for an example nuclear power plant (NPP) via a back-of-the-envelope calculation.

In [2]:
# import numpy as np

# # constants
# J_per_MeV = 1.60218e-13  # J/MeV
# J_per_GWd = 8.64e13  # J/GWd
# sec_per_year = 3.1536e7  # sec
# sec_per_day = 86400  # sec
# day_per_year = 365.25  # days
# L_per_m3 = 1e3  # L/m3
# i129_per_mol = 6.02e23  # avocado
# i129_yield = 0.706/100  # from https://www-nds.iaea.org/sgnucdat/c3.htm
# i129_halflife = 15.7e6*sec_per_year  # sec
# GBq_per_i129 = 1e-9*np.log(2)/i129_halflife  # GBq/atom I-129

# # assumptions
# burnup = 40  # GWd/MTIHM, chosen because seems average
# enrichment = 4  # %, chosen because seems average
# capacity_factor = 0.95
# thermal_eff = 0.35
# npp_power = 1e9  # W
# print(f"NPP parameters: burnup {burnup} GWd/MTIHM, enrichment {enrichment}%, capacity {capacity_factor}, efficiency {thermal_eff}, power production {npp_power/1e9} GWe-yr")

# # calculations
# npp_energy_perGWeyr = npp_power*sec_per_year*capacity_factor/thermal_eff  # J
# print(f"NPP thermal energy production per GWe-yr:", npp_energy_perGWeyr, "J")
# npp_fuel_perGWeyr = npp_energy_perGWeyr/(burnup*J_per_GWd)  # MTU
# print("NPP fuel required per GWe-yr:", npp_fuel_perGWeyr, "MTU")

### VERSION B: Copy reference PWR from ANL study

The cell below saves the reactor parameters from the PWR benchmark ("Ref. PWR") in the following study:
https://fuelcycleoptions.inl.gov/SiteAssets/SitePages/Home/SMR_Waste_Attributes_Report_Final.pdf

In [3]:
import numpy as np

# constants
J_per_MeV = 1.60218e-13  # J/MeV
J_per_GWd = 8.64e13  # J/GWd
sec_per_year = 3.1536e7  # sec
sec_per_day = 86400  # sec
day_per_year = 365.25  # days
L_per_m3 = 1e3  # L/m3
i129_per_mol = 6.02e23  # avocado
i129_yield = 0.706/100  # from https://www-nds.iaea.org/sgnucdat/c3.htm
i129_halflife = 15.7e6*sec_per_year  # sec
GBq_per_i129 = 1e-9*np.log(2)/i129_halflife  # GBq/atom I-129
GBq_per_pCi = 3.7e-11

# reactor parameters
burnup = 50  # GWd/MTIHM
enrichment = 4.5  # %
capacity_factor = 0.90
thermal_eff = 0.34
npp_power = 1.175e9  # W
npp_fuel_perGWeyr = 21.7  # MTU/GWe-yr

# Step 2: Calculate GBq I-129 in spent fuel after LLW and effluents

The cell below pulls the I-129 density in spent fuel from a subset of the Uppsala PWR spent fuel data library [1], then multiplies this by NPP fuel requirement to get I-129 generated per GWe-yr. This subset of the spent fuel data library ("filename" in the code below) was generated using i129_massFlow_readBurnupDataset.ipynb

Further information on the data in this PWR spent fuel data library, for example the fuel density, can be found in [2].

[1] https://data.mendeley.com/datasets/8z3smmw63p/1

[2] https://www.sciencedirect.com/science/article/pii/S2352340920313111

In [4]:
import pandas as pd

filename = 'burnup' + str(burnup) + '-enrich' + str(enrichment) + '-iodines' + '.csv'
# columns in .csv include:
# burnup BU in GWd/MTU, cooling time CT in days, enrichment IE in %
# TOT_SF Spontaneous fission rate in fissions per second on a per axial length (cm) basis
# TOT_A Activity in Becquerels on a per axial length basis
# the isotope atomic densities I127, I129, I131 given in units of 1e24/cm3
data = pd.read_csv(filename)

fuel_density = 10.5e-6  # MT/cm3, from PWR spent fuel data library paper https://www.sciencedirect.com/science/article/pii/S2352340920313111
i129_atomic_density = 1e24*data['I129']/fuel_density  # atom/MTU
i129_molar_density = i129_atomic_density/6.02e23  # mol/MTU
pwr_i129_atomic_density = np.average(i129_atomic_density)
pwr_i129_molar_density = np.average(i129_molar_density)

npp_i129_perGWeyr = pwr_i129_atomic_density*npp_fuel_perGWeyr  # atoms I-129
npp_i129_activity_perGWeyr = npp_i129_perGWeyr*GBq_per_i129  # GBq I-129
print("I-129 produced at NPP per GWe-yr:", npp_i129_activity_perGWeyr, "GBq")

loss_effluent_LLW = 0.1  # GBq/GWe-yr, loss to effluents + LLW shipments calculated separately as described in i129 paper
spentFuel_i129_activity_perGWeyr = npp_i129_activity_perGWeyr - loss_effluent_LLW  # GBq I-129
print("I-129 remaining in spent fuel per GWe-yr:", spentFuel_i129_activity_perGWeyr, "GBq")

I-129 produced at NPP per GWe-yr: 29.73631979326882 GBq
I-129 remaining in spent fuel per GWe-yr: 29.636319793268818 GBq


/Users/harukowainwright/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Reprocessing mass flow

In [23]:
# Estimate the La Hague filter efficiency
SNF2gas = 0.95 # % released as gas from SNF, Asmussen et al. (2022)
liquid_scrubbing = 0.96 # liquid scrubbing efficiency, Orano, 2021a, 2021b, 2022
liq2gas = 266.7 # liquid-to-gas release ratio

mass_gas = spentFuel_i129_activity_perGWeyr*SNF2gas*(1-liquid_scrubbing) 
mass_liq = spentFuel_i129_activity_perGWeyr*SNF2gas*liquid_scrubbing 

# filter efficiency
filter_efficiency = (mass_gas-(mass_liq/liq2gas))/mass_gas
filter_efficiency

0.9100112485939258

In [24]:
# Mass to LLW
mass_gas-(mass_liq/liq2gas)

1.0248366063944554

In [25]:
# Mass to the biosphere
mass_liq

27.02832365146116

In [26]:
# Purely based on gas filters
spentFuel_i129_activity_perGWeyr*SNF2gas*0.99

27.872958765569322